In [ ]:

import pandas as pd
import mysql.connector

# Read Excel files
df_5308 = pd.read_excel('5308.xls', sheet_name=None)
df_5329 = pd.read_excel('5329.xls', sheet_name=None)

# Print sheet names for verification
print("Sheet names in 5308.xls:", pd.ExcelFile('5308.xls').sheet_names)
print("Sheet names in 5329.xls:", pd.ExcelFile('5329.xls').sheet_names)


In [ ]:
def save_cell(cell_id, df, sheet_names, cursor):

    # Insert or update cell data
    insert_cells_query = """
    INSERT INTO cells (cell_id, discharge_capacity, nominal_capacity)
    VALUES (%s, %s, %s)
    ON DUPLICATE KEY UPDATE
    discharge_capacity = VALUES(discharge_capacity),
    nominal_capacity = VALUES(nominal_capacity)
    """
    cursor.execute(insert_cells_query, (
        cell_id,
        2992.02 if cell_id == 5308 else 2822.56,
        3000
    ))


In [ ]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='database3',
     connection_timeout=600  # Set timeout to 10 minutes

)

cursor = conn.cursor()


In [ ]:
def save_cell_data(cell_id, df, sheet_names, cursor, chunk_size=1000):
    # Determine the correct sheet names
    data_sheet = sheet_names[3]  # Adjust this index if needed
    temp_sheet = sheet_names[-1]  # Adjust this index if needed

    # Ensure the sheet names are correct
    if data_sheet not in df:
        print(f"Error: Data sheet '{data_sheet}' not found in DataFrame.")
        return
    if temp_sheet not in df:
        print(f"Error: Temperature sheet '{temp_sheet}' not found in DataFrame.")
        return

    # Check column indices
    if df[data_sheet].shape[1] <= 10:  # Adjust based on the actual number of columns needed
        print(f"Error: Data sheet '{data_sheet}' does not have enough columns.")
        return
    if df[temp_sheet].shape[1] <= 4:  # Adjust based on the actual number of columns needed
        print(f"Error: Temperature sheet '{temp_sheet}' does not have enough columns.")
        return

    # Insert or update cell data
    try:
        insert_cells_query = """
        INSERT INTO cells (cell_id, discharge_capacity, nominal_capacity)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE
        discharge_capacity = VALUES(discharge_capacity),
        nominal_capacity = VALUES(nominal_capacity)
        """
        cursor.execute(insert_cells_query, (
            cell_id,
            2992.02 if cell_id == 5308 else 2822.56,
            3000
        ))

        # Prepare and save cell data in chunks
        data = []
        for index, row in df[data_sheet].iterrows():
            if index < len(df[temp_sheet]):
                data.append((
                    cell_id,
                    row.iloc[5],  # Adjust column index based on actual data
                    row.iloc[6],
                    row.iloc[7],
                    df[temp_sheet].iloc[index, 4],  # Adjust column index based on actual data
                    pd.to_datetime(row.iloc[10], errors='coerce')  # Adjust column index based on actual data
                ))

            # Insert data in chunks
            if len(data) >= chunk_size:
                insert_cell_data_query = """
                INSERT INTO cell_data (cell_id, current_data, voltage_data, capacity_data, temperature_data, time_data)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
                cursor.executemany(insert_cell_data_query, data)
                data = []

        # Insert remaining data
        if data:
            insert_cell_data_query = """
            INSERT INTO cell_data (cell_id, current_data, voltage_data, capacity_data, temperature_data, time_data)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            cursor.executemany(insert_cell_data_query, data)

        print(f"Data for cell {cell_id} successfully inserted.")
        
    except Exception as e:
        print(f"Error processing cell {cell_id}: {e}")


In [ ]:
# Save data for both cells
save_cell(5308, df_5308, pd.ExcelFile('5308.xls').sheet_names, cursor)
save_cell(5329, df_5329, pd.ExcelFile('5329.xls').sheet_names, cursor)
save_cell_data(5329, df_5329, pd.ExcelFile('5329.xls').sheet_names, cursor)
save_cell_data(5308, df_5308, pd.ExcelFile('5308.xls').sheet_names, cursor)


# Commit the transaction and close the connection
conn.commit()
cursor.close()
conn.close()

print("Data successfully imported into MySQL database.")